In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os
import shutil
import logging
import subprocess
from copy import deepcopy
from time import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdDistGeom import EmbedMultipleConfs
from linux_qm.src.util import load_smiles3D, _create_tmp_dir, draw3Dconfs, SetPositions
from linux_qm.qm.crest.crest import conformer_pipeline
import pandas as pd

In [12]:
smi = 'COC1CN(C)C1'
# smi = 'CO'


# mol = load_smiles3D(smi)
# EmbedMultipleConfs(mol, numConfs=3)

mol = conformer_pipeline(smi)

# Get the conformers

conformers = mol.GetConformers()

# Print the number of conformers generated
print(f"Number of conformers generated: {len(conformers)}")
print("Energies:", [c.GetProp('energy') for c in conformers])

[28-11-2023 04:18:02] [INFO] Conformer generation pipeline
[28-11-2023 04:18:02] [DEBUG] SMILES parsing
[28-11-2023 04:18:02] [DEBUG] XTB pre-optimize geometry
[28-11-2023 04:18:02] [DEBUG] CREST generate conformers
[28-11-2023 04:18:04] [DEBUG] conformers: 5
[28-11-2023 04:18:04] [DEBUG] CREST conformer screen
[28-11-2023 04:18:05] [DEBUG] conformers: 4
[28-11-2023 04:18:05] [DEBUG] CREST clustering
[28-11-2023 04:18:05] [DEBUG] conformers: 4
[28-11-2023 04:18:05] [DEBUG] CREST Finished


Number of conformers generated: 4
Energies: ['-23.275472730000001', '-23.273513609999998', '-23.273084279999999', '-23.27281279']


In [4]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, autoalign=False)

(3, 2, 6, 4)
num conformers 3


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
import os
os.getcwd()

'/home/ergot/linux_qm/notebooks'

In [14]:
logging.getLogger().setLevel(logging.DEBUG)

In [21]:
from linux_qm.qm.orca.orca import OrcaDriver
orca = OrcaDriver()

conf = deepcopy(mol).GetConformer()

orca.options['n_jobs'] = 1
orca.options['method'] = 'HF-3c'
orca.single_point(conf)

[28-11-2023 04:31:07] [DEBUG] ORCA INPUT:
!HF-3c   
%geom MaxIter 100 end
%maxcore 2000
*xyz 0 1
C            2.71476633         -0.42888768          0.15407066
O            1.61468689          0.39695579          0.44574823
C            0.56782276          0.27299788         -0.47466517
C           -0.66126443          1.11992950         -0.13113815
N           -1.51499242         -0.04688400         -0.36976350
C           -2.58121789         -0.29509936          0.56447874
C           -0.36185900         -0.95000755         -0.33960068
H            3.46299374         -0.22813218          0.91809485
H            2.43961879         -1.48953798          0.18109183
H            3.12942997         -0.19830385         -0.83508249
H            0.93021833          0.41963947         -1.50069903
H           -0.88485953          1.96634447         -0.78037792
H           -0.62919213          1.44663215          0.91408794
H           -3.06192750         -1.23794527          0.30585955
H      

In [17]:
!java

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: line 1: java: command not found


In [13]:
orca.geometry_optimization(conf)

[25-11-2023 09:30:17] [INFO] Method: HF def2-SVP
[25-11-2023 09:30:17] [DEBUG] ORCA INPUT:
!HF def2-SVP OPT  
%geom MaxIter 100 end
%maxcore 2000
*xyz 0 1
C           -2.71481901         -0.42882777         -0.15404635
O           -1.61468760          0.39693510         -0.44575543
C           -0.56782240          0.27294017          0.47465200
C            0.66125083          1.11991196          0.13117504
N            1.51499833         -0.04689441          0.36976603
C            2.58125616         -0.29503597         -0.56445863
C            0.36188343         -0.95004097          0.33953245
H           -3.12947277         -0.19817433          0.83509541
H           -3.46303183         -0.22805518         -0.91808116
H           -2.43974061         -1.48949734         -0.18102236
H           -0.93021945          0.41951965          1.50069328
H            0.88481929          1.96630677          0.78045014
H            0.62918737          1.44665280         -0.91403957
H            

In [9]:
%%time

logging.getLogger().setLevel(logging.INFO)
orca.options['solvent'] = 'THF'

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-311++G(d,p) NMR'
orca.geometry_optimization(conf)

[24-11-2023 02:32:03] [INFO] Geometry optimization
[24-11-2023 02:32:03] [INFO] Method: M062X 6-311++G(d,p) NMR
[24-11-2023 02:34:15] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:34:15] [INFO] Success: True
[24-11-2023 02:34:15] [INFO] Num Iter: 9
[24-11-2023 02:34:15] [INFO] Elapsed Time: 131.8s


CPU times: user 126 ms, sys: 61.7 ms, total: 187 ms
Wall time: 2min 11s


In [10]:
%%time

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-31G'
orca.geometry_optimization(conf)

orca.options['method'] = 'M062X 6-311++G(d,p) NMR'
orca.geometry_optimization(conf)

[24-11-2023 02:34:15] [INFO] Geometry optimization
[24-11-2023 02:34:15] [INFO] Method: M062X 6-31G
[24-11-2023 02:35:07] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:35:07] [INFO] Success: True
[24-11-2023 02:35:07] [INFO] Num Iter: 9
[24-11-2023 02:35:07] [INFO] Elapsed Time: 52.0s
[24-11-2023 02:35:07] [INFO] Geometry optimization
[24-11-2023 02:35:07] [INFO] Method: M062X 6-311++G(d,p) NMR
[24-11-2023 02:36:48] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:36:48] [INFO] Success: True
[24-11-2023 02:36:48] [INFO] Num Iter: 7
[24-11-2023 02:36:48] [INFO] Elapsed Time: 101.1s


CPU times: user 243 ms, sys: 48.6 ms, total: 291 ms
Wall time: 2min 33s


In [11]:
%%time

conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVDZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 02:36:48] [INFO] Geometry optimization
[24-11-2023 02:36:48] [INFO] Method: M062X cc-pVDZ NMR
[24-11-2023 02:38:10] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:38:10] [INFO] Success: True
[24-11-2023 02:38:10] [INFO] Num Iter: 9
[24-11-2023 02:38:10] [INFO] Elapsed Time: 82.3s


CPU times: user 127 ms, sys: 48.6 ms, total: 175 ms
Wall time: 1min 22s


In [12]:
%%time
conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVTZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 02:38:10] [INFO] Geometry optimization
[24-11-2023 02:38:10] [INFO] Method: M062X cc-pVTZ NMR
[24-11-2023 02:41:28] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:41:28] [INFO] Success: True
[24-11-2023 02:41:28] [INFO] Num Iter: 9
[24-11-2023 02:41:28] [INFO] Elapsed Time: 198.2s


CPU times: user 175 ms, sys: 49.4 ms, total: 225 ms
Wall time: 3min 18s


In [13]:
%%time
conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X aug-cc-pVTZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 02:41:28] [INFO] Geometry optimization
[24-11-2023 02:41:28] [INFO] Method: M062X aug-cc-pVTZ NMR
[24-11-2023 02:50:45] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:50:45] [INFO] Success: True
[24-11-2023 02:50:45] [INFO] Num Iter: 9
[24-11-2023 02:50:45] [INFO] Elapsed Time: 556.3s


CPU times: user 195 ms, sys: 80.9 ms, total: 276 ms
Wall time: 9min 16s


In [17]:
%%time

conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVDZ'
orca.geometry_optimization(conf)

orca.options['method'] = 'M062X aug-cc-pVTZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 03:03:21] [INFO] Geometry optimization
[24-11-2023 03:03:21] [INFO] Method: M062X cc-pVDZ
[24-11-2023 03:04:42] [INFO] Identified logfile to be in ORCA format
[24-11-2023 03:04:42] [INFO] Success: True
[24-11-2023 03:04:42] [INFO] Num Iter: 9
[24-11-2023 03:04:42] [INFO] Elapsed Time: 80.5s
[24-11-2023 03:04:42] [INFO] Geometry optimization
[24-11-2023 03:04:42] [INFO] Method: M062X aug-cc-pVTZ NMR
[24-11-2023 03:10:10] [INFO] Identified logfile to be in ORCA format
[24-11-2023 03:10:10] [INFO] Success: True
[24-11-2023 03:10:10] [INFO] Num Iter: 5
[24-11-2023 03:10:10] [INFO] Elapsed Time: 328.7s


CPU times: user 234 ms, sys: 122 ms, total: 356 ms
Wall time: 6min 49s


In [15]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, confIds=[0], autoalign=False)

(3, 2, 6, 4)
num conformers 4


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [16]:
data.scfvalues

NameError: name 'data' is not defined

In [ ]:
data.scftargets

In [ ]:
# NMR shielding
for k,v in data.nmrtensors.items():
    print(mol.GetAtomWithIdx(k).GetSymbol(), k, v['isotropic'])

In [ ]:
dir(data)

In [ ]:
print(data.metadata['cpu_time'][0])

In [ ]:
len(data.converged_geometries)

In [ ]:
dir(data)

In [ ]:
data.optdone

In [ ]:
data.metadata

In [ ]:
data.new_geometries

In [ ]:
# HOMO LUMO
homo = data.moenergies[0][data.homos[0]]
lumo = data.moenergies[0][data.homos[0] + 1]
homo, lumo

In [ ]:
print(data.metadata['cpu_time'][0])

In [ ]:
data.natom, data.nmo

In [ ]:
data.metadata

In [ ]:
data.scfenergies